# Multiple Regression (gradient descent)

In this notebook we will cover estimating multiple regression weights via gradient descent. You will:
* Write a predict_output() function
* Compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [1]:
import pandas as pd

sales = pd.read_csv('./kc_house_data.csv')
sales.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [29]:
sales['intercept'] = np.ones(len(sales))
sales.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,intercept
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,1.0
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,1.0
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,1.0
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,1.0
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,1.0


In [30]:
import numpy as np

# Each row is a set of features for a particular sample.
example_features = sales[['intercept', 'sqft_living']].values

# Each element is a scalar output for a particular sample
example_output = sales['price'].values

print(example_features[0, :])
print(example_output[0])

[1.00e+00 1.18e+03]
221900.0


# Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0\*1.0 + 1.0\*1180.0 = 1181.0 this is the dot product between these two arrays. If they're numpy arrayws we can use np.dot() to compute this:

In [31]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0, :] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print(predicted_value)

1181.0


np.dot() also works when dealing with a matrix and a vector. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [32]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

If you want to test your code run the following cell:

In [34]:
test_predictions = predict_output(example_features, my_weights)
print(test_predictions[0]) # should be 1181.0
print(test_predictions[1]) # should be 2571.0

1181.0
2571.0


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [35]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative = 2 * np.dot(errors, feature)
    return(derivative)

To test your feature derivartive run the following:

In [38]:
example_features = sales[['intercept', 'sqft_living']].values
example_output = sales['price'].values

my_weights = np.array([0., 0.]) # this makes all the predictions 0

test_predictions = predict_output(example_features, my_weights) 
errors = test_predictions - example_output

feature = example_features[:,0] # let's compute the derivative with respect to 'intercept'
derivative = feature_derivative(errors, feature)

print(derivative)
print(-np.sum(example_output)*2) # should be the same as derivative

-23345850016.0
-23345850016.0


# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature before computing our stopping criteria

In [39]:
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [65]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array

    while not converged:
        predictions = predict_output(feature_matrix, weights)
        errors = predictions - output

        gradient_sum_squares = 0
        for i in range(len(weights)):
            feature_column = feature_matrix[:, i]
            derivative_i = feature_derivative(errors, feature_column)
            gradient_sum_squares += (derivative_i * derivative_i)
            weights[i] = weights[i] - step_size * derivative_i

        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)

        if gradient_magnitude < tolerance:
            converged = True

    return(weights)

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

# Running the Gradient Descent as Simple Regression

First let's split the data into training and test data.

In [62]:
train_mask = np.random.rand(len(sales)) < 0.8

train_data = sales[train_mask]
test_data = sales[~train_mask]

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimat the parameters in the simple regression on squarefeet. The folowing cell sets up the feature_matrix, output, initial weights and step size for the first model:

In [63]:
# let's test out the gradient descent
simple_features = ['intercept', 'sqft_living']
my_output = 'price'

simple_feature_matrix = train_data[simple_features].values
output = train_data[my_output].values

initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

Next run your gradient descent with the above parameters.

In [66]:
simple_feature_weights = regression_gradient_descent(
    simple_feature_matrix,
    output,
    initial_weights,
    step_size,
    tolerance)

simple_feature_weights

array([-46999.88780859,    282.91596092])

Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [68]:
test_simple_feature_matrix = test_data[simple_features].values
test_output = test_data[my_output].values

Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [69]:
predictions = predict_output(test_simple_feature_matrix, simple_feature_weights)
predictions[0:5]

array([507515.39558716, 456590.52262225, 465078.00144973, 306645.06333666,
       349082.45747409])

Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [70]:
error = predictions - test_output
rss = np.dot(errors, errors)
rss

9217325138472070.0

# Running a multiple regression

Now we will use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [71]:
model_features = ['intercept', 'sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'

feature_matrix = train_data[model_features].values
output = train_data[my_output].values

initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

Use the above parameters to estimate the model weights. Record these values for your quiz.

In [73]:
weights = regression_gradient_descent(
    feature_matrix,
    output,
    initial_weights,
    step_size,
    tolerance)

weights

array([-9.99999813e+04,  2.46122856e+02,  6.52073532e+01])

Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [78]:
test_feature_matrix = test_data[model_features].values
test_output = test_data[my_output].values

predictions = predict_output(test_feature_matrix, weights)
predictions[0:5]

array([471082.81723361, 454167.79145803, 434164.38879856, 291119.00105126,
       365857.69434572])

What is the actual price for the 1st house in the test data set vs the predicted price?

In [79]:
print('Predicted first house price:', predictions[0])
print('Actual first house price:', test_output[0])

Predicted first house price: 471082.8172336102
Actual first house price: 604000.0


Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [80]:
errors = predictions - test_output
rss = np.dot(errors, errors)
rss

267547176386135.8